In [10]:
import json
data = []

data_type = "dev"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 141


In [11]:
ner_tag = set([
    "PER",   # player
    "CLU",   # club
    "TME",   # time
    "NUM",   # number 
])

relation_tag = set([
    "COMP",  # (CLU, CLU) compete with
    "DEFE",  # (CLU, CLU) defeat / win over
    # "SCON",  # (CLU, NUM) score
    "SCOP",  # (CLU, PER) score player
    "SCOT",  # (PER, TME) score time
    "CARP",  # (CLU, PER) card player,
    "CART",  # (PER, TIME) card time
    "SUBP",  # (PER, PER) substitute players,
    "SUBT",  # (PER, TIME) substitute time
])

In [12]:
from vncorenlp import VnCoreNLP
segmentor = VnCoreNLP("../../../VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg")

Entity format:

```
<entity id> <ner_tag> <start_pos> <end_pos> <text>
```

Ví dụ:
```
T3-7	GPE 522 531	community
```

In [13]:
class Entity():
    global_id = 1
    def __init__(self, ner_type, start_pos, end_pos, text):
        self.id = f"T-{Entity.global_id}"
        self.ner_type = ner_type
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.text = text
        Entity.global_id += 1
        
    def __repr__(self):
        return f"{self.id}\t{self.ner_type} {self.start_pos} {self.end_pos}\t{self.text}"
        
class Relation():
    global_id = 1
    def __init__(self, relation_type, *args):
        self.id = f"R-{Relation.global_id}"
        self.relation_type = relation_type
        self.args = args
        Relation.global_id += 1
        
    def __repr__(self):
        res = f"{self.id}\t{self.relation_type}"
        for args_id, entity in enumerate(self.args):
            res += f" Arg{args_id + 1}:{entity.id}"
        return res
    
def segmentize(text):
    text = segmentor.tokenize(text.strip())
    return " ".join(" ".join(x) for x in text)

In [14]:
from bs4 import BeautifulSoup

def parse_paragraph(data):
    event_texts = {0: ""}
    event_paddings = {0: 0}
    event_ids = []
    e_id = None
    content = ""
    
    for html in data["html_annotation"]:
        for e in BeautifulSoup(html):
            try:
                e_id = int(e["event_id"])
                event_ids.append(e_id)
                event_texts[e_id] = ""
            except: pass
            if e_id is None:
                continue
            event_texts[e_id] += segmentize(e.text) + " "
    event_ids.sort()
    
    for i, event in enumerate(event_ids):
        cur_event_ids = event_ids[i]
        pre_event_ids = event_ids[i - 1] if i != 0 else 0
        event_paddings[cur_event_ids] = event_paddings[pre_event_ids] + len(event_texts[pre_event_ids])
        content += event_texts[cur_event_ids]
    
    event_texts["0"] = content
    title = segmentize(data["original_doc"]["_source"]["description"])
    return {
        "title": title,
        "content": content,
        "event_texts": event_texts,
        "event_paddings": event_paddings,
    }

In [15]:
def is_match(team1, team2):
    team1 = team1.strip()
    team2 = team2.strip()
    return team1 in team2 or team2 in team1

In [16]:
def full_text_token(text, start, passage):
    end = start + len(text)
    while end < len(passage) and passage[end] != " ":
        end += 1
    while start > 0 and passage[start - 1] != " ":
        start -= 1
    text = passage[start : end]
    return start, text

In [17]:
def parse_entity_relation(data, event_texts, event_paddings):
    entities, relations = [], []
    entities_id = set()
    def add_entity(entity):
        if entity.start_pos == entity.end_pos:
            return
        entities_id.add(entity.id)
        entities.append(entity)
    
    def add_relation(relation):
        for entity in relation.args:
            if entity.id not in entities_id:
                return
        relations.append(relation)
    
    def parse_entity_other(ner_type, text, ref_event_ids):
        text = segmentize(text)
        for eid in ref_event_ids.split(","):
            pos = event_texts[int(eid)].lower().find(text.lower())
            if pos != -1:
                pos, text = full_text_token(text, pos, event_texts[int(eid)])
                start_pos = pos + event_paddings[int(eid)]
                end_pos = start_pos + len(text)
                return Entity(ner_type, start_pos, end_pos, text)
            
        edited_texts = []
        if ner_type == "PER" and len(text.split()) != 1:
            edited_texts = text.split()
        elif ner_type == "TME" and "'" in text:
            text = text.replace("'", "")
            edited_texts = ["phút thứ " + text, "phút " + text] 
        for text in edited_texts:
            edited_ner = parse_entity_other(ner_type, text, ref_event_ids)
            if edited_ner.start_pos != edited_ner.end_pos:
                return edited_ner
            
        return Entity(ner_type, -1, -1, text)
    
    def parse_entity_score(ner_type, score1, score2, ref_event_ids):
        edited_scores = [score1 + "-" + score2, score1 + " - " + score2, score2 + "-" + score1, score2 + " - " + score1]
        for eid in ref_event_ids.split(","):
            for score in edited_scores:
                score = segmentize(score)
                pos = event_texts[int(eid)].find(score)
                if pos != -1:
                    pos, score = full_text_token(score, pos, event_texts[int(eid)])
                    start_pos = event_paddings[int(eid)] + pos
                    end_pos = start_pos + len(score)
                    return Entity(ner_type, start_pos, end_pos, score)
        return Entity(ner_type, -1, -1, score1 + "-" + score2)
    
    
    
    id = data["train_id"]
    summary = data["match_summary"]
    team = summary["players"]
    score_board = summary["score_board"]
    score_list = summary["score_list"]
    card_list = summary["card_list"]
    subst_list = summary["substitution_list"]
    
    # Yield teams' name
    team1_ner = parse_entity_other("CLU", team["team1"], team["ref_event_ids"])
    team2_ner = parse_entity_other("CLU", team["team2"], team["ref_event_ids"])
    add_entity(team1_ner)
    add_entity(team2_ner)
    add_relation(Relation("COMP", team1_ner, team2_ner))
    
    # Yield scores
    score_ner = parse_entity_score("SCO", score_board["score1"], score_board["score2"], score_board["ref_event_ids"])
    # score_ner = parse_entity_score("SCO", score_board["score1"], score_board["score2"], "0")
    add_entity(score_ner)
    if int(score_board["score1"]) > int(score_board["score2"]):
        add_relation(Relation("DEFE", team1_ner, team2_ner))
    elif int(score_board["score1"]) < int(score_board["score2"]):
        add_relation(Relation("DEFE", team2_ner, team1_ner))
    else:
        add_relation(Relation("DRAW", team1_ner, team2_ner))
    
    # Yield time
    for info in score_list:
        player = parse_entity_other("PER", info["player_name"], info["ref_event_ids"])
        time = parse_entity_other("TME", info["time"], info["ref_event_ids"])
        team = team1_ner if info["team"] == team1_ner.text else team2_ner
        add_entity(player)
        add_entity(time)
        add_relation(Relation("SCOP", team, player))
        add_relation(Relation("SCOT", player, time))
        
        
    for info in card_list:
        player = parse_entity_other("PER", info["player_name"], info["ref_event_ids"])
        time = parse_entity_other("TME", info["time"], info["ref_event_ids"])
        team = team1_ner if is_match(info["team"], team1_ner.text) else team2_ner
        add_entity(player)
        add_entity(time)
        add_relation(Relation("CARP", team, player))
        add_relation(Relation("CART", player, time))
        
    for info in subst_list:
        player_in = parse_entity_other("PER", info["player_in"], info["ref_event_ids"])
        player_out = parse_entity_other("PER", info["player_out"], info["ref_event_ids"])
        add_entity(player_in)
        add_entity(player_out)
        add_entity(time)
        add_relation(Relation("SUBP", player_out, player_in))
        if "time" in info:
            time = parse_entity_other("TME", info["time"], info["ref_event_ids"])
            add_entity(time)
            add_relation(Relation("SUBT", player_in, time))
        
    return entities, relations

In [17]:
sample_idx = 1
paragraph = parse_paragraph(data[sample_idx])
content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
entities, relations = parse_entity_relation(data[sample_idx], event_texts, event_paddings)

print(*entities, sep="\n")
print()
print(*relations, sep="\n")

T-25	CLU 113 128	Atletico_Madrid
T-26	CLU 203 208	Betis
T-27	SCO 43 46	1-0
T-28	PER 1790 1796	Correa
T-29	TME 1650 1652	74

R-17	COMP Arg1:T-25 Arg2:T-26
R-18	DEFE Arg1:T-25 Arg2:T-26
R-19	SCOP Arg1:T-26 Arg2:T-28
R-20	SCOT Arg1:T-28 Arg2:T-29


In [18]:
from tqdm import tqdm

if data_type == "train":
    data[346]["match_summary"]["card_list"] = []
    data[375]["match_summary"]["score_list"][1]["ref_event_ids"] = "3"

for x in tqdm(data):
    file_id = x["train_id"]
    para = parse_paragraph(x)
    entities, relations = parse_entity_relation(x, para["event_texts"], para["event_paddings"])
    
    with open(f"./{data_type}/{file_id}.txt", "w") as f:
        # f.write(para["title"] + "\n")
        f.write(para["content"])
        
    with open(f"./{data_type}/{file_id}.ann", "w") as f:
        for entity in entities:
            f.write(str(entity) + "\n")
        for relation in relations:
            f.write(str(relation) + "\n")

100%|██████████| 141/141 [00:18<00:00,  7.51it/s]


In [39]:
para["content"]

'Sài_Gòn FC là đội nhỉnh hơn với nhiều cơ_hội nguy_hiểm được tạo ra . Tuy_nhiên các cầu_thủ lại không_thể_nào tận_dụng được cơ_hội và bất_ngờ thua trước khi Thành Nguyện của Tân_Hiệp Hưng dứt_điểm chính_xác sau tình_huống phản_công nhanh . Ngày_mai , Tân_Hiệp Hưng sẽ đụng_độ nhà đương_kim vô_địch - Thái_Sơn_Nam . Sau bàn thua , các cầu_thủ Sài_Gòn FC đẩy cao đội_hình và liên_tiếp dồn_ép đối_phương . Và những nỗ_lực không biết mệt_mỏi của họ đã được đền_đáp khi Hoàng_Minh đánh gót tinh_tế gỡ hoà 1-1 . Đây cũng là kết_quả cuối_cùng của trận đấu . Sau trận đấu , HLV Nguyễn_Bảo_Trung của Sài_Gòn FC tỏ ra tiếc_nuối khi các cầu_thủ không tận_dụng được cơ_hội và để thua trước . Ông nói : “ Bên đội Sài_Gòn có rất nhiều cơ_hội nhưng các cầu_thủ không tận_dụng được , rồi thua trước nên cuối_cùng phải cố_gắng lội ngược ” . Trong khi đó HLV Phạm_Minh_Giang của Tân_Hiệp Hưng cho rằng thế_trận của trận đấu là năm – năm , cơ_hội chia đều cho cả hai đội và đội nào tận_dụng cơ_hội tốt hơn sẽ giành chiế